In [9]:
import numpy as np
import cv2

  
  
# Capturing video through webcam
webcam = cv2.VideoCapture(0)
webcam.set(3,640)
webcam.set(4,480)

# Define Trackbar On Change Value
def do_nothing(x):
    pass

# Init Empty Black Image To Describe Lower limit and Upper limit for Color Detection
L_img = np.zeros((480, 640, 3), np.uint8)
U_img = np.zeros((480, 640, 3), np.uint8)

# Init Windows
cv2.namedWindow("Res",cv2.WINDOW_NORMAL)
cv2.namedWindow("HSV Control",cv2.WINDOW_NORMAL)

# Init Trackbar for Setting Lower Limit and Upper Limit Of HSV Detection
cv2.createTrackbar("Lower Hue","HSV Control"        ,28,179,do_nothing)
cv2.createTrackbar("Lower Saturation","HSV Control" ,245,255,do_nothing)
cv2.createTrackbar("Lower Value","HSV Control"      ,150,255,do_nothing)
cv2.createTrackbar("Upper Hue","HSV Control"        ,34,179,do_nothing)
cv2.createTrackbar("Upper Saturation","HSV Control" ,255,255,do_nothing)
cv2.createTrackbar("Upper Value","HSV Control"      ,255,255,do_nothing)

# Define Windows Initial Position
cv2.moveWindow("Res",50,100)
cv2.moveWindow("HSV Control",475,100)

# Get Image Background
_, backgroundFrame = webcam.read()

# Read Stream
while(1):
      
    # Reading the video from the
    # webcam in image frames
    _, imageFrame = webcam.read()
    
    imageFrame = cv2.absdiff(imageFrame,backgroundFrame)
    
    # Assign Trackbar value to variable
    L_H = cv2.getTrackbarPos("Lower Hue","HSV Control")
    L_S = cv2.getTrackbarPos("Lower Saturation","HSV Control")
    L_V = cv2.getTrackbarPos("Lower Value","HSV Control")    
    
    U_H = cv2.getTrackbarPos("Upper Hue","HSV Control")
    U_S = cv2.getTrackbarPos("Upper Saturation","HSV Control")
    U_V = cv2.getTrackbarPos("Upper Value","HSV Control")
    
    # Assert Boundaries
    if L_H > U_H:
        L_H = U_H
    if L_S > U_S:
        L_S = U_S
    if L_V > U_V:
        L_V = U_V
        
        
    L_limit=np.array([L_H,L_S,L_V]) # setting the lower limit
    U_limit=np.array([U_H,U_S,U_V]) # setting the upper limit
        
    # Convert Image To HSV
    into_hsv =cv2.cvtColor(imageFrame,cv2.COLOR_BGR2HSV)
    
    # Blur to reduce Noise
    into_hsv = cv2.blur(into_hsv,(7,7))
    
    # Create Mask
    b_mask=cv2.inRange(into_hsv,L_limit,U_limit)
    
    
    # Reduce Mask Splitting on same region
    kernel = np.ones((5, 5), np.uint8)
    b_mask = cv2.dilate(b_mask, kernel, iterations=3)

    # creating the mask using inRange() function
    # this will produce an image where the color of the objects
    # falling in the range will turn white and rest will be black
    blue=cv2.bitwise_and(imageFrame,imageFrame,mask=b_mask)
    
    
    L_img[:] = [L_H,L_S,L_V]
    U_img[:] = [U_H,U_S,U_V]
    L_img = cv2.cvtColor(L_img,cv2.COLOR_HSV2BGR)
    U_img = cv2.cvtColor(U_img,cv2.COLOR_HSV2BGR)
    outputControl = np.concatenate((L_img,U_img),axis=1)
    
    contours, hierarchy = cv2.findContours(b_mask,
                                           cv2.RETR_TREE,
                                           cv2.CHAIN_APPROX_SIMPLE)
    for pic, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if(area > 300):
            x, y, w, h = cv2.boundingRect(contour)
            imageFrame = cv2.rectangle(imageFrame, (x, y),
                                       (x + w, y + h),
                                       (255, 0, 0), 2)
              
            cv2.putText(imageFrame, "Detected Object", (x, y),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1.0, (255, 0, 0))
    outputFrame = np.concatenate((imageFrame,blue),axis=1)
    
    
    cv2.imshow("Res", outputFrame)
    cv2.imshow("HSV Control",outputControl)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        webcam.release()
        cv2.destroyAllWindows()
        break
